In [1]:
import pandas as pd
from sklearn.utils import shuffle

# Loading Data

## pandas.read_csv

In [2]:
file_path = "../../data/raw/"

X = pd.read_csv(f"{file_path}x_train_gr_smpl.csv", delimiter=',')
X.head()

,0,1,2,3,4,5,6,7,8,9,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,30.0,29.0,28.0,29.0,31.0,30.0,29.0,28.0,27.0,26.0,...,31.0,32.0,35.0,38.0,39.0,39.0,40.0,39.0,39.0,38.0
1,31.0,31.0,33.0,32.0,31.0,30.0,29.0,28.0,28.0,28.0,...,32.0,34.0,35.0,36.0,36.0,37.0,38.0,38.0,37.0,37.0
2,30.0,30.0,31.0,29.0,28.0,27.0,26.0,28.0,30.0,31.0,...,33.0,35.0,37.0,37.0,38.0,39.0,38.0,38.0,39.0,40.0
3,26.0,25.0,24.0,24.0,24.0,27.0,28.0,29.0,29.0,30.0,...,32.0,34.0,36.0,37.0,38.0,42.0,40.0,37.0,36.0,36.0
4,25.0,26.0,28.0,28.0,28.0,28.0,28.0,27.0,26.0,25.0,...,30.0,31.0,33.0,37.0,38.0,37.0,36.0,36.0,35.0,35.0


In [3]:
Y = pd.read_csv(f"{file_path}y_train_smpl.csv", delimiter=',')
Y.columns = ['target']

y0 = pd.read_csv(f"{file_path}y_train_smpl_0.csv", delimiter=',')
y0.columns = ['target']

y1 = pd.read_csv(f"{file_path}y_train_smpl_1.csv", delimiter=',')
y1.columns = ['target']

y2 = pd.read_csv(f"{file_path}y_train_smpl_2.csv", delimiter=',')
y2.columns = ['target']

y3 = pd.read_csv(f"{file_path}y_train_smpl_3.csv", delimiter=',')
y3.columns = ['target']

y4 = pd.read_csv(f"{file_path}y_train_smpl_4.csv", delimiter=',')
y4.columns = ['target']

y5 = pd.read_csv(f"{file_path}y_train_smpl_5.csv", delimiter=',')
y5.columns = ['target']

y6 = pd.read_csv(f"{file_path}y_train_smpl_6.csv", delimiter=',')
y6.columns = ['target']

y7 = pd.read_csv(f"{file_path}y_train_smpl_7.csv", delimiter=',')
y7.columns = ['target']

y8 = pd.read_csv(f"{file_path}y_train_smpl_8.csv", delimiter=',')
y8.columns = ['target']

y9 = pd.read_csv(f"{file_path}y_train_smpl_9.csv", delimiter=',')
y9.columns = ['target']

Y.tail()

,target
12655,9
12656,9
12657,9
12658,9
12659,9


# Create datasets

In [4]:
train_smpl_0 = pd.concat([X, y0], axis=1)
train_smpl_1 = pd.concat([X, y1], axis=1)
train_smpl_2 = pd.concat([X, y2], axis=1)
train_smpl_3 = pd.concat([X, y3], axis=1)
train_smpl_4 = pd.concat([X, y4], axis=1)
train_smpl_5 = pd.concat([X, y5], axis=1)
train_smpl_6 = pd.concat([X, y6], axis=1)
train_smpl_7 = pd.concat([X, y7], axis=1)
train_smpl_8 = pd.concat([X, y8], axis=1)
train_smpl_9 = pd.concat([X, y9], axis=1)

In [5]:
train_smpl = pd.concat([X, Y], axis=1)
train_smpl.head()

,0,1,2,3,4,5,6,7,8,9,...,2295,2296,2297,2298,2299,2300,2301,2302,2303,target
0,30.0,29.0,28.0,29.0,31.0,30.0,29.0,28.0,27.0,26.0,...,32.0,35.0,38.0,39.0,39.0,40.0,39.0,39.0,38.0,0
1,31.0,31.0,33.0,32.0,31.0,30.0,29.0,28.0,28.0,28.0,...,34.0,35.0,36.0,36.0,37.0,38.0,38.0,37.0,37.0,0
2,30.0,30.0,31.0,29.0,28.0,27.0,26.0,28.0,30.0,31.0,...,35.0,37.0,37.0,38.0,39.0,38.0,38.0,39.0,40.0,0
3,26.0,25.0,24.0,24.0,24.0,27.0,28.0,29.0,29.0,30.0,...,34.0,36.0,37.0,38.0,42.0,40.0,37.0,36.0,36.0,0
4,25.0,26.0,28.0,28.0,28.0,28.0,28.0,27.0,26.0,25.0,...,31.0,33.0,37.0,38.0,37.0,36.0,36.0,35.0,35.0,0


# class distribution

In [ ]:
%matplotlib inline
train_smpl.hist(column='target')

In [ ]:
train_smpl.target.value_counts()

# Data Randomisation

## sklear.utils.suffle

In [6]:
train_smpl = shuffle(train_smpl, random_state=42)
train_smpl_0 = shuffle(train_smpl_0, random_state=42)
train_smpl_1 = shuffle(train_smpl_1, random_state=42)
train_smpl_2 = shuffle(train_smpl_2, random_state=42)
train_smpl_3 = shuffle(train_smpl_3, random_state=42)
train_smpl_4 = shuffle(train_smpl_4, random_state=42)
train_smpl_5 = shuffle(train_smpl_5, random_state=42)
train_smpl_6 = shuffle(train_smpl_6, random_state=42)
train_smpl_7 = shuffle(train_smpl_7, random_state=42)
train_smpl_8 = shuffle(train_smpl_8, random_state=42)
train_smpl_9 = shuffle(train_smpl_9, random_state=42)
train_smpl.head()

,0,1,2,3,4,5,6,7,8,9,...,2295,2296,2297,2298,2299,2300,2301,2302,2303,target
1236,25.0,24.0,28.0,27.0,24.0,22.0,22.0,22.0,21.0,21.0,...,34.0,33.0,34.0,36.0,35.0,35.0,34.0,34.0,35.0,0
8847,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,241.0,230.0,237.0,227.0,241.0,241.0,239.0,241.0,237.0,5
11285,34.0,34.0,34.0,33.0,32.0,31.0,31.0,30.0,30.0,30.0,...,16.0,14.0,12.0,11.0,11.0,11.0,13.0,17.0,19.0,8
12423,63.0,66.0,70.0,81.0,77.0,72.0,62.0,61.0,60.0,62.0,...,123.0,77.0,47.0,44.0,43.0,42.0,55.0,71.0,68.0,9
10020,38.0,39.0,40.0,50.0,61.0,72.0,93.0,138.0,185.0,233.0,...,62.0,68.0,68.0,81.0,98.0,86.0,73.0,68.0,65.0,6


In [ ]:
train_smpl.info()

In [ ]:
train_smpl.describe()

# Reducing the size

# Features/Attributes Selection

## correlation matrix

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#plt.matshow(train_smpl.corr())
#plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
print(train_smpl.shape)

best_20 = SelectKBest(chi2, k=20).fit_transform(train_smpl[train_smpl.columns[:2303]], train_smpl['target'])
best_20 = pd.DataFrame(best_20)
train_smpl_20 = pd.concat([best_20, train_smpl['target']], axis=1)
print(train_smpl_20.shape)

best_50 = SelectKBest(chi2, k=50).fit_transform(train_smpl[train_smpl.columns[:2303]], train_smpl['target'])
best_50 = pd.DataFrame(best_50)
train_smpl_50 = pd.concat([best_50, train_smpl['target']], axis=1)
print(train_smpl_50.shape)

best_100 = SelectKBest(chi2, k=100).fit_transform(train_smpl[train_smpl.columns[:2303]], train_smpl['target'])
best_100 = pd.DataFrame(best_100)
train_smpl_100 = pd.concat([best_100, train_smpl['target']], axis=1)
print(train_smpl_100.shape)

# Test Train Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_smpl[train_smpl.columns[:2303]], train_smpl['target'], test_size=0.33, random_state=42)

X_train_20, X_test_20, y_train_20, y_test_20 = train_test_split(train_smpl_20[train_smpl_20.columns[:20]], train_smpl_20['target'], test_size=0.33, random_state=42)
X_train_50, X_test_50, y_train_50, y_test_50 = train_test_split(train_smpl_50[train_smpl_50.columns[:50]], train_smpl_50['target'], test_size=0.33, random_state=42)
X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(train_smpl_100[train_smpl_100.columns[:100]], train_smpl_100['target'], test_size=0.33, random_state=42)


X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(train_smpl_0[train_smpl_0.columns[:2303]], train_smpl_0['target'], test_size=0.33, random_state=42)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(train_smpl_1[train_smpl_1.columns[:2303]], train_smpl_1['target'], test_size=0.33, random_state=42)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(train_smpl_2[train_smpl_2.columns[:2303]], train_smpl_2['target'], test_size=0.33, random_state=42)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(train_smpl_3[train_smpl_3.columns[:2303]], train_smpl_3['target'], test_size=0.33, random_state=42)
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(train_smpl_4[train_smpl_4.columns[:2303]], train_smpl_4['target'], test_size=0.33, random_state=42)
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(train_smpl_5[train_smpl_5.columns[:2303]], train_smpl_5['target'], test_size=0.33, random_state=42)
X_train_6, X_test_6, y_train_6, y_test_6 = train_test_split(train_smpl_6[train_smpl_6.columns[:2303]], train_smpl_6['target'], test_size=0.33, random_state=42)
X_train_7, X_test_7, y_train_7, y_test_7 = train_test_split(train_smpl_7[train_smpl_7.columns[:2303]], train_smpl_7['target'], test_size=0.33, random_state=42)
X_train_8, X_test_8, y_train_8, y_test_8 = train_test_split(train_smpl_8[train_smpl_8.columns[:2303]], train_smpl_8['target'], test_size=0.33, random_state=42)
X_train_9, X_test_9, y_train_9, y_test_9 = train_test_split(train_smpl_9[train_smpl_9.columns[:2303]], train_smpl_9['target'], test_size=0.33, random_state=42)

X_train_10classes = [X_train_0, X_train_1, X_train_2, X_train_3, X_train_4, X_train_5, X_train_6, X_train_7, X_train_8, X_train_9]
X_test_10classes = [X_test_0, X_test_1, X_test_2, X_test_3, X_test_4, X_test_5, X_test_6, X_test_7, X_test_8, X_test_9]
y_train_10classes = [y_train_0, y_train_1, y_train_2, y_train_3, y_train_4, y_train_5, y_train_6, y_train_7, y_train_8, y_train_9]
y_test_10classes = [y_test_0, y_test_1, y_test_2, y_test_3, y_test_4, y_test_5, y_test_6, y_test_7, y_test_8, y_test_9]

# Modeling

## Multinomial Naive Bayes model (multi-class classifier)

### Before Features/Attributes Selection

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
plot_confusion_matrix(conf_mat, target_names=y_test.unique())

In [ ]:
# looking for devious class labels that take high numbers of misclassifications
for i in range(conf_mat.shape[1]):
    column = conf_mat.T[i]
    misclassifications = column.sum() - column[i]
    print(misclassifications)

### After Features/Attributes Selection (20/50/100)

In [ ]:
X_trains_reduced = [  X_train_20, X_train_50, X_train_100]
X_tests_reduced = [ X_test_20, X_test_50, X_test_100]
y_trains_reduced = [ y_train_20, y_train_50, y_train_100]
y_tests_reduced = [ y_test_20, y_test_50, y_test_100]

for X_trains, X_tests, y_trains, y_tests in zip(X_trains_reduced, X_tests_reduced, y_trains_reduced, y_tests_reduced):    
    clf = MultinomialNB()
    clf.fit(X_trains, y_trains)
    print(clf.score(X_tests, y_tests))

## Gaussian Naive Bayes model (mono-class classifier)

In [ ]:
from sklearn.naive_bayes import GaussianNB
scores = []
for X_train, X_test, y_train, y_test in zip(X_train_10classes, X_test_10classes, y_train_10classes, y_test_10classes):    
    clf = GaussianNB()
    clf.fit(X_train, y_train)
    scores.append(clf.score(X_test, y_test))
    print(clf.score(X_test, y_test))
print("mean: ", sum(scores)/len(scores))

## Conclusions from initial experiments (q7)

Using the multilabel confusion matrix, it is apparent that the multinomial Naïve Bayes classifier struggled with some classes, predicting them incorrectly most of the time. 

In particular, class 8 was more often classified as class 1 (probability 0.278), class 7 (probability 0.2023) and class 6 (probability 0.1974) before itself (probability 0.1283).

//It looks like the classifier was confused by ...



The strongest misclassification in the matrix is true members of class 3 being classified as class 7 (probability 0.2023). However, there is relatively very little of the reverse misclassification (true class 7 predicted as class 3).

//^^ investigate this?



Class 7 punches above its weight as the most abundant misclassification which is perhaps due to the relative infrequency of data for class 7. There must be some other factor, as there are less populated classes.



## Clustering

In [13]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler(feature_range=[0, 1])
# data_rescaled = scaler.fit_transform(train_smpl_0[train_smpl_0.columns[:-1]])

#pca = PCA(n_components=250)
# training = pd.DataFrame(pca.fit_transform(train_smpl_1))
# training = pd.DataFrame(data_rescaled)
training = pd.DataFrame(train_smpl_1)
training["target"] = train_smpl_1["target"]

kmeans = KMeans(n_clusters=2, random_state=2)
training["cluster"] = kmeans.fit_predict(training[training.columns[:-1]])

In [ ]:
weight = [1] * 251
training["cluster_class_attribute"] = kmeans.fit_predict(training)
training

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#Fitting the PCA algorithm with our Data
pca = PCA().fit(data_rescaled)

#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Pulsar Dataset Explained Variance')
plt.show()

In [11]:
result = training[training.columns[-3:]]
# target = training[training.columns[-3:-1]]

# print(target.groupby(['target']).count())
result_cluster = result.groupby(['target', 'cluster']).count()
# result_class_attribute = result.groupby(['target', 'cluster_class_attribute']).count()
# result_cluster["class"] = result_class_attribute["cluster"]
result_cluster

2303
target cluster      
0      0         389
       1        1471
1      0        4395
       1        6405